In [ ]:
pip install sentence-transformers

     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 2.3MB 37.9MB/s 
     |████████████████████████████████| 1.2MB 45.6MB/s 
     |████████████████████████████████| 901kB 54.1MB/s 
     |████████████████████████████████| 3.3MB 51.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-cp37-none-any.whl size=123339 sha256=2ae3152e8f9704947c228100b054887acf5942f40d9b5d9f2ce928b94fad956c
  Stored in directory: /root/.cache/pip/wheels/0f/06/f7/faaa96fdda87462b4fd5c47b343340e9d5531ef70d0eef8242
Successfully built sentence-transformers


In [ ]:
import pandas as pd
import os 
import ast
import sentence_transformers 
import matplotlib.pyplot as plt 
import pickle
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens') 

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-large-nli-stsb-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
interactions = pd.read_csv('/content/drive/MyDrive/archive/RAW_interactions.csv')


In [ ]:
grouping= interactions.groupby(['recipe_id']).agg({'rating' : ['mean'],'user_id' : ['nunique']}).reset_index()
grouping.head(5)

,recipe_id,rating,user_id
,,mean,nunique
0,38,4.250000,4
1,39,3.000000,1
2,40,4.333333,9
3,41,4.500000,2
4,43,1.000000,1


In [ ]:
grouping.columns = ['recipe_id', 'rating_mean', 'user_num']
grouping.columns = pd.Index([e[0] + "_" +e[1] for e in grouping.columns.tolist()])


In [ ]:
rei = grouping[ (grouping['user_id_nunique'] > 5)]

In [ ]:
recipes = pd.read_csv('/content/drive/MyDrive/archive/RAW_recipes.csv')
filter_recipe = pd.merge(recipes,rei,right_on = ['recipe_id'],left_on = ['id'],how = 'inner')

In [ ]:
filter_recipe.shape

(42884, 15)

In [ ]:
encodings_recipe_df = pickle.load(open("/content/drive/MyDrive/recipe_embedding.pickle",'rb'))
print ("Encoding are loaded")
data_encoding = pd.DataFrame(encodings_recipe_df)

Encoding are loaded


In [ ]:

encoding_sparse = sparse.csr_matrix(encodings_recipe_df[0:10000])

In [ ]:
#also can output sparse matrices
import datetime
time = datetime.datetime.now()
print (time)
similarities_sparse = cosine_similarity(encoding_sparse)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print ("Time taken is :",)
print (datetime.datetime.now()-time)

2021-05-27 15:07:39.403938
Time taken is :
0:03:36.273101


In [ ]:
### Convert the pickle file to datafarme and dump
df1 = pd.DataFrame(similarities_sparse)
pickle.dump(df1,open('similarities_sparse.pickle','wb'))

In [ ]:
data_similarity = df1.unstack().reset_index()
data_similarity.columns = ['recipe1','recipe2','cosine_similarity']

In [ ]:
data_similarity = data_similarity[data_similarity['cosine_similarity']<0.9999]
data_similarity = data_similarity[data_similarity['cosine_similarity']>0.7]
print (data_similarity.shape)

(23306414, 3)


In [ ]:
recipe_dict = {}
for j,i in enumerate(filter_recipe['name']):
    recipe_dict[j] = i


In [ ]:
data_similarity['recipe1_name'] = data_similarity['recipe1'].map(recipe_dict)
data_similarity['recipe2_name'] = data_similarity['recipe2'].map(recipe_dict)

In [ ]:
data_similarity['similarity_rank'] = data_similarity.groupby(['recipe1'])['cosine_similarity'].rank("dense", ascending=False)
data_similarity = data_similarity[data_similarity['similarity_rank'] <= 4].reset_index()

In [ ]:
data_similarity

,index,recipe1,recipe2,cosine_similarity,recipe1_name,recipe2_name,similarity_rank
0,1087,0,1087,0.824837,better then bush s baked beans,angel s rich and buttery pan rolls bread machine,3.0
1,1090,0,1090,0.824706,better then bush s baked beans,angelic shrimp and lobster,4.0
2,5816,0,5816,0.833334,better then bush s baked beans,broccoli chicken casserole,2.0
3,6352,0,6352,0.835143,better then bush s baked beans,butterscotch ride,1.0
4,10392,1,392,0.862677,chicken lickin good pork chops,500f eye of round roast,3.0
...,...,...,...,...,...,...,...
39554,99988315,9998,8315,0.794778,cinnamon nuts,chicken enchilada bake with salsa cream sauce,4.0
39555,99990398,9999,398,0.895510,cinnamon oatmeal,6 week bran muffins auntie anne s muffins,2.0
39556,99993401,9999,3401,0.891251,cinnamon oatmeal,banberry jam,3.0
39557,99997445,9999,7445,0.887514,cinnamon oatmeal,cheddar potato soup,4.0


In [ ]:
def find_similar_dishes(recipe):
    data =  data_similarity[data_similarity['recipe1_name'] == recipe]
    print ("You should try ")
    data.sort_values(inplace = True,by =['similarity_rank']) 
    for j,i in enumerate(data['recipe2_name']):
       print (" ", i)
       

In [ ]:
find_similar_dishes('cinnamon nuts')

You should try 
  biscuit stuffing pork chops
  cafe orange mocha
  betty crocker peach pancakes
  chicken enchilada bake with salsa cream sauce


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
find_similar_dishes(['cinnamon nuts','birdseed bread'])

In [ ]:
data_similarity

,index,recipe1,recipe2,cosine_similarity,recipe1_name,recipe2_name,similarity_rank
0,1087,0,1087,0.824837,arriba baked winter squash mexican style,acorn squash a la kree,3.0
1,1090,0,1090,0.824706,arriba baked winter squash mexican style,acorn squash bisque,4.0
2,1101,0,1101,0.817159,arriba baked winter squash mexican style,acorn squash with pineapple,5.0
3,5816,0,5816,0.833334,arriba baked winter squash mexican style,baked delicata squash with lime butter,2.0
4,6352,0,6352,0.835143,arriba baked winter squash mexican style,baked spaghetti squash with jalapeno cream,1.0
...,...,...,...,...,...,...,...
49958,99990398,9999,398,0.895510,birdseed bread,100 whole wheat bread non dense heavy white...,2.0
49959,99993401,9999,3401,0.891251,birdseed bread,arabian pita bread,3.0
49960,99994138,9999,4138,0.885125,birdseed bread,auberge chorizo goat s cheese and onion pizza...,5.0
49961,99997445,9999,7445,0.887514,birdseed bread,barbari a persian bread,4.0
